In [ ]:
print('Insert average berth time of ports')

In [2]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math
import string

In [3]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [4]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [5]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [6]:
# Accommodate query
sql = """SELECT id_port, code, port_name
FROM stg_pelni.master_port
where flag_ppss like 'Y'"""

In [7]:
# Read CSV
df = pd.read_csv('data/mean.csv')

In [8]:
# Read Query
df1 = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_14068\2371418704.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = sqlio.read_sql_query(sql, conn)


In [9]:
# Grab data in faktur (column and row) 
cleandf = df[['Unnamed: 0', 'berth_time', 'berth_time.1', 'berth_time.2']][2:95]
cleandf

,Unnamed: 0,berth_time,berth_time.1,berth_time.2
2,CIWANDAN,1.11,3.2,1.715
3,GILIMAS,176.02,176.02,176.02
4,P.MARE,29.240000000000002,29.240000000000002,29.240000000000002
5,Agats,0.54,8.33,2.5009574468085107
6,Ambon,1.29,16.54,4.246474576271186
...,...,...,...,...
83,Tual,1.18,6.4,3.101843971631206
84,Waikelo,0.35,3.17,1.24
85,Waingapu,1.0,10.07,2.8221505376344083
86,Wanci,0.47,24.24,2.6201886792452833


In [10]:
# Change columns name
cleandf = cleandf.rename(columns={'Unnamed: 0' : 'ke'})
cleandf = cleandf.rename(columns={'berth_time' : 'min'})
cleandf = cleandf.rename(columns={'berth_time.1' : 'max'})
cleandf = cleandf.rename(columns={'berth_time.2' : 'mean'})

In [11]:
# Change columns name
df1 = df1.rename(columns={'port_name' : 'ke'})

# Inner join
cleandf['ke'] = cleandf['ke'].astype(str)
df1['ke'] = df1['ke'].astype(str)

result = pd.merge(df1, 
                      cleandf, 
                      on ='ke', 
                      how ='inner')
result

,id_port,code,ke,min,max,mean
0,00739854-654f-469c-b5af-31680f7ae36e,662,Labuan Bajo,0.48,5.42,1.876045197740113
1,ddac0441-65ff-4960-b993-87c73ebc2d85,322,Belinyu,0.3,2.18,0.8021153846153847
2,9c1e061d-9b9f-404d-a4cc-c98db3337a64,951,Namlea,1.01,7.06,2.075454545454545
3,979da725-6d99-47af-9e0d-f1650ff74414,395,Panjang,8.41,22.23,15.0525
4,3349ad05-b76a-47d2-ae53-83e0b2b457ab,891,Awerange/Barru,0.0,3.26,1.2690909090909093
...,...,...,...,...,...,...
70,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,947,Banda Naira,0.14,33.51,8.271911764705882
71,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,979,Merauke,1.27,37.51,14.3335593220339
72,72346087-6a6a-462b-a434-913d6f2e5ca9,942,Ternate,0.36,7.29,2.1357386363636364
73,1334ec95-cc8a-4464-a263-0b0beee559da,251,Sei Kolak Kijang,0.49,11.31,2.272061855670103


In [12]:
# Change Typedata
result['min'] = result['min'].astype(float)
result['max'] = result['max'].astype(float)
result['mean'] = result['mean'].astype(float)


In [13]:
# Used function math.ceil (min)
ceil_min = []
for index, row in result.iterrows():
    a = row['min']
    rs = math.ceil(a)
    ceil_min.append(rs)
    
result['min'] = ceil_min    

# Used function math.ceil (max)
ceil_max = []
for index, row in result.iterrows():
    a = row['max']
    rs = math.ceil(a)
    ceil_max.append(rs)
    
result['max'] = ceil_max    


# Used function math.ceil (mean) 
ceil_mean = []
for index, row in result.iterrows():
    a = row['mean']
    rs = math.ceil(a)
    ceil_mean.append(rs)
    
result['mean'] = ceil_mean    


In [14]:
# Change typedata
result['min'] = result['min'].astype(int)
result['max'] = result['max'].astype(int)
result['mean'] = result['mean'].astype(int)

# reset index
result.index = np.arange(1, len(result) + 1)


In [15]:
result

,id_port,code,ke,min,max,mean
1,00739854-654f-469c-b5af-31680f7ae36e,662,Labuan Bajo,1,6,2
2,ddac0441-65ff-4960-b993-87c73ebc2d85,322,Belinyu,1,3,1
3,9c1e061d-9b9f-404d-a4cc-c98db3337a64,951,Namlea,2,8,3
4,979da725-6d99-47af-9e0d-f1650ff74414,395,Panjang,9,23,16
5,3349ad05-b76a-47d2-ae53-83e0b2b457ab,891,Awerange/Barru,0,4,2
...,...,...,...,...,...,...
71,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,947,Banda Naira,1,34,9
72,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,979,Merauke,2,38,15
73,72346087-6a6a-462b-a434-913d6f2e5ca9,942,Ternate,1,8,3
74,1334ec95-cc8a-4464-a263-0b0beee559da,251,Sei Kolak Kijang,1,12,3


In [16]:
result = result[result['ke'] != "Belinyu"]

In [17]:
result

,id_port,code,ke,min,max,mean
1,00739854-654f-469c-b5af-31680f7ae36e,662,Labuan Bajo,1,6,2
3,9c1e061d-9b9f-404d-a4cc-c98db3337a64,951,Namlea,2,8,3
4,979da725-6d99-47af-9e0d-f1650ff74414,395,Panjang,9,23,16
5,3349ad05-b76a-47d2-ae53-83e0b2b457ab,891,Awerange/Barru,0,4,2
6,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,Ambon,2,17,5
...,...,...,...,...,...,...
71,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,947,Banda Naira,1,34,9
72,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,979,Merauke,2,38,15
73,72346087-6a6a-462b-a434-913d6f2e5ca9,942,Ternate,1,8,3
74,1334ec95-cc8a-4464-a263-0b0beee559da,251,Sei Kolak Kijang,1,12,3


In [18]:
# result.to_csv('data/min_max_mean.csv')